Scraping from Web

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil ulasan dari satu halaman
def scrape_reviews(page_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    review_containers = soup.find_all('div', class_='lister-item-content')

    for container in review_containers:
        title = container.find('a', class_='title').text.strip() if container.find('a', class_='title') else 'No title'
        username = container.find('span', class_='display-name-link').text.strip() if container.find('span', class_='display-name-link') else 'No username'
        review_text = container.find('div', class_='text').text.strip() if container.find('div', class_='text') else 'No review'
        date = container.find('span', class_='review-date').text.strip() if container.find('span', class_='review-date') else 'No date'

        titles.append(title)
        usernames.append(username)
        reviews.append(review_text)
        dates.append(date)

# Inisialisasi List untuk menyimpan data ulasan
titles = []
usernames = []
reviews = []
dates = []

# Base URL untuk ulasan IMDb
base_url = 'https://www.imdb.com/title/tt3783958/reviews/_ajax?ref_=undefined&paginationKey={}'

# Mulai scrapping dari halaman pertama
pagination_key = ''
while True:
    page_url = base_url.format(pagination_key)
    scrape_reviews(page_url)

    # Untuk menememukan kunci pagination  halaman berikutnya
    soup = BeautifulSoup(requests.get(page_url).content, 'html.parser')
    pagination_key = soup.find('div', class_='load-more-data')['data-key'] if soup.find('div', class_='load-more-data') else None

    # Hentikan loop jika tidak ada lagi ulasan yang ditemukan
    if not pagination_key:
        break

    time.sleep(2)  # Untuk menghindari terlalu banyak permintaan ke server IMDb


# membuat DataFrame dari data tersebut
df = pd.DataFrame({
    'Title': titles,
    'Username': usernames,
    'Review': reviews,
    'Date': dates
})

# yang terakhir, # Simpan ke dalam CSV


df.to_csv('all_imdb_reviews.csv', index=False)

print(f"Scraped {len(df)} reviews!")

Scraped 2331 reviews!
